In [1]:
from difflib import SequenceMatcher
from scipy.sparse import lil_matrix
from sklearn.preprocessing import normalize
import csv
import cPickle
import fuzzy
import os
import re

from calcSimScore import *
from string_based_cluster import *
from meta_path import *
from post_process import *
from custom_setting import *
from name import *


In [4]:
(name_instance_dict, id_name_dict, name_statistics, author_paper_stat, metapaths) = load_files()

	Serialization files related to name_statistics do not exist.
	Reading in the author.csv file.
	Reading in the paperauthor.csv file.
	Writing into serialization files related to name_statistics.
	Serialization files related to authors do not exist.
	Reading in the author.csv file.
	Start recovering names
	Writing into serialization files related to name_statistics.

	Writing into serialization files related to authors.

	Serialization files related to coauthors do not exist.
	Reading in the paperauthor.csv file.
	Computing the coauthor graph.
	Writing into serialization files related to coauthors.

	Serialization files related to author_venue do not exist.
	Reading in the sanitizedPaper.csv file.
	Computing the author_venue matrix.
	Computing the covenue matrix.
	Writing into serialization files related to author_venue.

	Serialization files related to author_word do not exist.
	Reading in the sanitizedPaper.csv file and roughly filter words.
	There are in totoal 1637 words.
	Computing

In [5]:
add_similar_ids_under_name(name_instance_dict, id_name_dict)

	Building virtual names.
	Adding similar author ids into each name instance.
	Building reduced_name dict.
	Adding similar ids for the same reduced_names.
	Building sorted name dict.
	Adding similar ids for the same sorted_names.
	Building name unit dict.
	Adding similar ids for the same name units.
	Building name initials mapping.
	Start noisy last name comparison:
	Start question marks or non askii name comparison:
		In total there exist 83 names containing question marks or non askii characters.
	Start arbitrary name comparison:

	Start nicknames comparison:


In [6]:
cPickle.dump(name_instance_dict, open(serialization_dir + "step2_" + name_instance_file, "wb"), 2)

In [ ]:
name_instance_dict = cPickle.load(open(serialization_dir + 'step2_' + name_instance_file, "rb"))

In [7]:
potential_duplicate_groups = create_potential_duplicate_groups(name_instance_dict, author_paper_stat)

In [9]:
len(potential_duplicate_groups)

86269

In [10]:
similarity_score_dict = dict()
real_duplicate_groups1 = local_clustering(similarity_score_dict, potential_duplicate_groups, author_paper_stat,
                                              name_instance_dict, id_name_dict, name_statistics, metapaths)

	Finish computing similarities for 0.0% (0/86269) possible duplicate groups.
	Statistic about merges based on different features: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	Finish computing similarities for 11.5916493758% (10000/86269) possible duplicate groups.
	Statistic about merges based on different features: [0, 0, 96, 0, 5785, 0, 3262, 0, 0, 0, 0, 0, 0, 0]
	Finish computing similarities for 23.1832987516% (20000/86269) possible duplicate groups.
	Statistic about merges based on different features: [0, 0, 215, 0, 11624, 0, 6443, 0, 0, 0, 0, 0, 0, 0]
	Finish computing similarities for 34.7749481274% (30000/86269) possible duplicate groups.
	Statistic about merges based on different features: [0, 0, 335, 0, 17383, 0, 9701, 0, 0, 0, 0, 0, 0, 0]
	Finish computing similarities for 46.3665975032% (40000/86269) possible duplicate groups.
	Statistic about merges based on different features: [0, 0, 450, 0, 23194, 0, 12907, 0, 0, 0, 0, 0, 0, 0]
	Finish computing similarities for 57.958246

In [12]:
real_duplicate_groups2 = local_clustering(similarity_score_dict, potential_duplicate_groups, author_paper_stat,
                                          name_instance_dict, id_name_dict, name_statistics, metapaths)


	Finish computing similarities for 0.0% (0/86269) possible duplicate groups.
	Statistic about merges based on different features: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	Finish computing similarities for 11.5916493758% (10000/86269) possible duplicate groups.
	Statistic about merges based on different features: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	Finish computing similarities for 23.1832987516% (20000/86269) possible duplicate groups.
	Statistic about merges based on different features: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	Finish computing similarities for 34.7749481274% (30000/86269) possible duplicate groups.
	Statistic about merges based on different features: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	Finish computing similarities for 46.3665975032% (40000/86269) possible duplicate groups.
	Statistic about merges based on different features: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
	Finish computing similarities for 57.9582468789% (50000/86269) possible duplic

In [13]:
real_duplicate_groups = real_duplicate_groups1.union(real_duplicate_groups2)

In [18]:
real_duplicate_groups

{(1651, 1724),
 (381, 392),
 (609, 621),
 (1626, 1718),
 (425, 438),
 (1760, 1772),
 (104, 115),
 (747, 795),
 (489, 575),
 (1503, 1504),
 (1478, 1522),
 (1113, 1118),
 (468, 611),
 (1724, 1738),
 (1560, 1563),
 (454, 586),
 (751, 800),
 (1673, 1820),
 (1546, 1554),
 (1629, 1731),
 (1046, 1089),
 (1389, 1418),
 (1426, 1428),
 (481, 601),
 (1664, 1782),
 (467, 538),
 (506, 508),
 (515, 552),
 (466, 479),
 (1430, 1447),
 (1649, 1723),
 (1635, 1656),
 (1461, 1536),
 (1432, 1543),
 (791, 796),
 (1529, 1541),
 (1524, 1528),
 (445, 448),
 (528, 572),
 (1501, 1527),
 (514, 611),
 (622, 640),
 (952, 1004),
 (883, 896),
 (666, 678),
 (337, 379),
 (1652, 1752),
 (1641, 1813),
 (1685, 1739),
 (1627, 1734),
 (1126, 1155),
 (1163, 1175),
 (338, 390),
 (170, 185),
 (469, 556),
 (1659, 1769),
 (861, 890),
 (455, 521),
 (741, 772),
 (1453, 1474),
 (192, 207),
 (1428, 1460),
 (879, 906),
 (588, 613),
 (1674, 1784),
 (496, 597),
 (1623, 1743),
 (482, 536),
 (468, 475),
 (1579, 1581),
 (664, 712),
 (1360

In [14]:
cPickle.dump(real_duplicate_groups, open(serialization_dir + "step4_" + "real_duplicate_groups", "wb"), 2)

In [15]:
authors_duplicates_dict = merge_local_clusters(real_duplicate_groups, id_name_dict)
find_closure(authors_duplicates_dict)
refine_result(authors_duplicates_dict, name_instance_dict, id_name_dict, name_statistics, similarity_score_dict,
              metapaths, True)
iter_num = 2
while iter_num > 0:
    find_closure(authors_duplicates_dict)
    refine_result(authors_duplicates_dict, name_instance_dict, id_name_dict, name_statistics, similarity_score_dict,
                  metapaths, True)
    iter_num -= 1

	Mapping each author to his/her duplicate authors from duplicate groups.
	Finding close duplicate author set for each author id.
		Iteration 0
	Removing 0 author_ids from name comparison.
	Finding who are really duplicates among the conflict author_ids
	Sorting conflicted author pairs according to similarity scores.
	Finding close duplicate author set for each author id.
		Iteration 0
	Removing 0 author_ids from name comparison.
	Finding who are really duplicates among the conflict author_ids
	Sorting conflicted author pairs according to similarity scores.
	Finding close duplicate author set for each author id.
		Iteration 0
	Removing 0 author_ids from name comparison.
	Finding who are really duplicates among the conflict author_ids
	Sorting conflicted author pairs according to similarity scores.


In [16]:
final_filter(author_paper_stat, name_statistics, authors_duplicates_dict, name_instance_dict, id_name_dict,
                 similarity_score_dict, metapaths)

	Removing 0 author_ids from name comparison.
	Finish removing 0.0 unconfident names.


In [17]:
"""Generate the submission file and fullname file for analysis."""
directory = os.path.dirname(duplicate_authors_file)
if not os.path.exists(directory):
    os.makedirs(directory)
with open("duplicate_authors_file.csv", 'wb') as result_file:
    result_file.write("AuthorId,DuplicateAuthorIds" + '\n')
    for author_id in sorted(authors_duplicates_dict.iterkeys()):
        result_file.write(str(author_id) + ',' + str(author_id))
        id_list = sorted(authors_duplicates_dict[author_id])
        for id in id_list:
            result_file.write(' ' + str(id))
        result_file.write('\n')

with open("duplicate_authors_full_name_file.csv", 'wb') as result_file:
    for author_id in sorted(authors_duplicates_dict.iterkeys()):
        result_file.write(id_name_dict[author_id][1]
                          + ' ' + str(author_id))
        id_list = sorted(authors_duplicates_dict[author_id])
        for id in id_list:
            result_file.write(', ' + id_name_dict[id][1] + ' ' + str(id))
        result_file.write('\n')